Comparing scaling behaviour for routing between pytket 0.4.1 and qiskit 0.13.0 as in section 6.2.2 in https://arxiv.org/pdf/2002.09783.pdf <br>

This notebook is currently set up only to compare scaling for the Rochester device with the 'benchmark for scalability study' circuits. This can easily be changed by changing the 'device_name' and 'device_n_qubits' to other devices.<br>
Method:<br>
• Create pytket circuit object from qasm and pre-optimise it using 'Transform::OptimisePostRouting'.<br>
• For qiskit, convert to qiskit qobj, route circuit using PassManager with DenseLayout and StochasticSwap, convert back to pytket circuit object.<br>
• For pytket, place circuit using 'GraphPlacement' and route circuit using generic route method.<br>
• Decompose SWAP gates to CX gates ( & bridge gates for pytket case), convert to IBM primitives, post-optimise (remove redundancies) using 'Transform::OptimisePostRouting'.<br>
• Compare depth of input and output circuits.

In [2]:
from pytket.qasm import circuit_from_qasm
from pytket.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket import Device, Architecture, Transform
from pytket.routing import GraphPlacement, route
from CONNECTION import INDEX_CONNECTION_LIST as idl
from qiskit.transpiler import CouplingMap, Layout, PassManager
from qiskit.transpiler.passes import StochasticSwap, DenseLayout
import os
import time

In [3]:
device_name = 'Rochester'
device_n_qubits = 53

In [4]:
# create tket device
tket_device = Device(Architecture(idl[device_name]))
placer = GraphPlacement(tket_device)
placer.modify_config(**dict(depth_limit = 30))
# create qiskit pass manager
qisk_coupling = CouplingMap(idl[device_name])
dl = DenseLayout(coupling_map = qisk_coupling)
ss = StochasticSwap(coupling_map = qisk_coupling)
pm = PassManager([dl,ss])

In [ ]:
repo_directory = os.getcwd()
res = []
count = 0
for filename in os.listdir(repo_directory + '/BSS/'):
    qbs = int(filename[:2])
    optimal_depth = int(filename[6:9])
    number = int(filename[17])
    if qbs is device_n_qubits:
        circ = circuit_from_qasm(repo_directory + '/BSS/' + filename)
        depth_before = circ.depth()
#         using a tket optimisation pass to clean up circuit pre routing significantly improves depth ratio
#         Transform.OptimisePostRouting().apply(circ)
        c = circ.copy()
        
#         convert pre-optimised circuit to qiskit object
        qisk_circ = tk_to_qiskit(circ)
        qisk_start = time.time()
#         map qiskit circuit with DenseLayout and StochasticSwap passes
        qisk_out = pm.run(qisk_circ)
        qisk_end = time.time()
#         convert back to tket object
        circ_after = qiskit_to_tk(qisk_out)
#         clean up circuit and rebase to qiskit device primitives using pytket
        Transform.RebaseToQiskit().DecomposeSWAPtoCX().apply(circ_after)
        qisk_depth_after = circ_after.depth()
        qisk_depth_ratio = qisk_depth_after/depth_before
        
        tk_start = time.time()
        placer.place(c)
#         route circuit in tket
        routed_circ = route(c, tket_device)
        tk_end = time.time()
#         clean up circuit and rebase to qiskit device primitives using pytket
        Transform.RebaseToQiskit().DecomposeSWAPtoCX().DecomposeBRIDGE().apply(routed_circ)
        tk_depth_after = routed_circ.depth()
        tk_depth_ratio = tk_depth_after/depth_before
        res.append([filename, 'tket', qbs, optimal_depth, number, depth_before, qisk_depth_after, qisk_depth_ratio, qisk_end-qisk_start, tk_depth_after, tk_depth_ratio, tk_end-tk_start])
        print(filename, ' | Circuit Depth Pre Routing: ', depth_before, ' | Qiskit Results, depth after: ', 
              qisk_depth_after, ', depth ratio:', qisk_depth_ratio,  ', time taken: ', qisk_end - qisk_start,
             ' | Pytket results, depth after: ', tk_depth_after, ', depth ratio: ', tk_depth_ratio, ', time taken: ', tk_end-tk_start)

53QBT_600CYC_QSE_5.qasm  | Circuit Depth Pre Routing:  600  | Qiskit Results, depth after:  5563 , depth ratio: 9.271666666666667 , time taken:  111.56923913955688  | Pytket results, depth after:  600 , depth ratio:  1.0 , time taken:  0.2707540988922119
53QBT_100CYC_QSE_4.qasm  | Circuit Depth Pre Routing:  100  | Qiskit Results, depth after:  1341 , depth ratio: 13.41 , time taken:  23.967076063156128  | Pytket results, depth after:  100 , depth ratio:  1.0 , time taken:  0.05911993980407715
53QBT_300CYC_QSE_6.qasm  | Circuit Depth Pre Routing:  300  | Qiskit Results, depth after:  3426 , depth ratio: 11.42 , time taken:  63.108272075653076  | Pytket results, depth after:  300 , depth ratio:  1.0 , time taken:  0.17036700248718262
53QBT_400CYC_QSE_7.qasm  | Circuit Depth Pre Routing:  400  | Qiskit Results, depth after:  4018 , depth ratio: 10.045 , time taken:  76.80331897735596  | Pytket results, depth after:  400 , depth ratio:  1.0 , time taken:  0.1659259796142578
53QBT_700CYC_Q